# CME538 Big Project - Data Hustlers

In [1]:
# Import 3rd party libraries
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt
import seaborn as sns

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

In [2]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium

## Import Weather Data

In [4]:
weather_files = os.listdir()
weather_filenames = [file for file in weather_files if 'P1D' in file]

# Print file names
print(weather_filenames[0:5])

['en_climate_daily_ON_6158355_2018_P1D.csv', 'en_climate_daily_ON_6158355_2019_P1D.csv', 'en_climate_daily_ON_6158355_2020_P1D.csv', 'en_climate_daily_ON_6158355_2021_P1D.csv', 'en_climate_daily_ON_6158355_2017_P1D.csv']


In [5]:
weather_data = []

for file in weather_filenames:
    data = pd.read_csv(file)
    weather_data.append(data)
    
weather_data = pd.concat(weather_data)

# View DataFrame
weather_data.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-79.4,43.67,TORONTO CITY,6158355,2018-01-01,2018,1,1,NaN,-7.9,...,NaN,M,0.2,NaN,11.0,NaN,NaN,M,NaN,M
1,-79.4,43.67,TORONTO CITY,6158355,2018-01-02,2018,1,2,NaN,-7.1,...,NaN,M,0.0,NaN,9.0,NaN,NaN,M,NaN,M
2,-79.4,43.67,TORONTO CITY,6158355,2018-01-03,2018,1,3,NaN,-5.3,...,NaN,M,0.2,NaN,11.0,NaN,NaN,M,NaN,M
3,-79.4,43.67,TORONTO CITY,6158355,2018-01-04,2018,1,4,NaN,-7.7,...,NaN,M,0.2,NaN,11.0,NaN,NaN,M,NaN,M
4,-79.4,43.67,TORONTO CITY,6158355,2018-01-05,2018,1,5,NaN,-14.7,...,NaN,M,0.0,NaN,10.0,NaN,NaN,M,NaN,M


In [6]:
# Converting to Date/Time Index
weather_data['Date/Time'] = pd.to_datetime(weather_data['Date/Time'])
weather_data.set_index(pd.DatetimeIndex(weather_data['Date/Time']), inplace=True)

weather_data.drop(columns=['Date/Time'], inplace=True)

weather_data.index = weather_data.index.tz_localize('EST')

# View DataFrame
weather_data.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
Date/Time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00-05:00,-79.4,43.67,TORONTO CITY,6158355,2018,1,1,NaN,-7.9,NaN,...,NaN,M,0.2,NaN,11.0,NaN,NaN,M,NaN,M
2018-01-02 00:00:00-05:00,-79.4,43.67,TORONTO CITY,6158355,2018,1,2,NaN,-7.1,NaN,...,NaN,M,0.0,NaN,9.0,NaN,NaN,M,NaN,M
2018-01-03 00:00:00-05:00,-79.4,43.67,TORONTO CITY,6158355,2018,1,3,NaN,-5.3,NaN,...,NaN,M,0.2,NaN,11.0,NaN,NaN,M,NaN,M
2018-01-04 00:00:00-05:00,-79.4,43.67,TORONTO CITY,6158355,2018,1,4,NaN,-7.7,NaN,...,NaN,M,0.2,NaN,11.0,NaN,NaN,M,NaN,M
2018-01-05 00:00:00-05:00,-79.4,43.67,TORONTO CITY,6158355,2018,1,5,NaN,-14.7,NaN,...,NaN,M,0.0,NaN,10.0,NaN,NaN,M,NaN,M


## Data Cleaning

In [7]:
# Display Column Names
print(weather_data.columns)

Index(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID', 'Year',
       'Month', 'Day', 'Data Quality', 'Max Temp (°C)', 'Max Temp Flag',
       'Min Temp (°C)', 'Min Temp Flag', 'Mean Temp (°C)', 'Mean Temp Flag',
       'Heat Deg Days (°C)', 'Heat Deg Days Flag', 'Cool Deg Days (°C)',
       'Cool Deg Days Flag', 'Total Rain (mm)', 'Total Rain Flag',
       'Total Snow (cm)', 'Total Snow Flag', 'Total Precip (mm)',
       'Total Precip Flag', 'Snow on Grnd (cm)', 'Snow on Grnd Flag',
       'Dir of Max Gust (10s deg)', 'Dir of Max Gust Flag',
       'Spd of Max Gust (km/h)', 'Spd of Max Gust Flag'],
      dtype='object')


In [8]:
# Remove the Flag columns, as they are not relevant for our analysis.
weather_data = weather_data.drop(['Longitude (x)',
                                  'Latitude (y)',
                                  'Climate ID',
                                  'Min Temp Flag', 
                                  'Max Temp Flag', 
                                  'Mean Temp Flag', 
                                  'Heat Deg Days Flag', 
                                  'Cool Deg Days Flag', 
                                  'Total Rain Flag', 
                                  'Total Snow Flag', 
                                  'Total Precip Flag', 
                                  'Snow on Grnd Flag', 
                                  'Dir of Max Gust Flag', 
                                  'Spd of Max Gust Flag'], axis=1)

In [9]:
# Verify that columns were removed
print(weather_data.columns)

Index(['Station Name', 'Year', 'Month', 'Day', 'Data Quality', 'Max Temp (°C)',
       'Min Temp (°C)', 'Mean Temp (°C)', 'Heat Deg Days (°C)',
       'Cool Deg Days (°C)', 'Total Rain (mm)', 'Total Snow (cm)',
       'Total Precip (mm)', 'Snow on Grnd (cm)', 'Dir of Max Gust (10s deg)',
       'Spd of Max Gust (km/h)'],
      dtype='object')


In [10]:
# Check for missing values.
missing_weather = weather_data.isnull().sum()

weather_data_missing = pd.DataFrame(missing_weather, columns=['count'])
weather_data_missing.index.name = 'Column Name'

# View DataFrame
weather_data_missing

,count
Column Name,
Station Name,0
Year,0
Month,0
Day,0
Data Quality,2556
Max Temp (°C),38
Min Temp (°C),29
Mean Temp (°C),40
Heat Deg Days (°C),40


We will remove only the rows where the max, min, and mean temperature data is missing, as these are key metrics which we'll use in our later analysis. We will not remove missing value for the precipitation columns, as that just incidates that there was no precipitation, not necessarily a sign of "missing" data.

In [11]:
# Remove rows with missing values in max_temp, min_temp, and mean_temp columns
weather_data = weather_data.dropna(subset=['Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)'])

# Check the updated DataFrame
print(weather_data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2018-01-01 00:00:00-05:00 to 2022-12-31 00:00:00-05:00
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Station Name               2516 non-null   object 
 1   Year                       2516 non-null   int64  
 2   Month                      2516 non-null   int64  
 3   Day                        2516 non-null   int64  
 4   Data Quality               0 non-null      float64
 5   Max Temp (°C)              2516 non-null   float64
 6   Min Temp (°C)              2516 non-null   float64
 7   Mean Temp (°C)             2516 non-null   float64
 8   Heat Deg Days (°C)         2516 non-null   float64
 9   Cool Deg Days (°C)         2516 non-null   float64
 10  Total Rain (mm)            0 non-null      float64
 11  Total Snow (cm)            0 non-null      float64
 12  Total Precip (mm)          2511 non-null   float64
 13  

In [12]:
# Add a 'season' column for seasonality analysis
def assign_season(row):
    if row['Month'] in [12, 1, 2]:
        return 'Winter'
    elif row['Month'] in [3, 4, 5]:
        return 'Spring'
    elif row['Month'] in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

weather_data['Season'] = weather_data.apply(assign_season, axis=1)

In [13]:
# Save the merged DataFrame as csv
weather_data.to_csv('weather_df.csv', index=False)

The saved CSV file was then moved to the "clean_data" folder.